In [ ]:
"""
Created on Mon Oct 21 17:55 2024

Prepare the Davison patterns for easier implementation in IPSL-CM

@author: Clara Burgard
"""

In [2]:
import xarray as xr
import numpy as np
import multimelt.useful_functions as uf
from cdo import Cdo
from tqdm.notebook import tqdm

In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_mask_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [ ]:
img = xr.open_dataset(inputpath_raw + 'basal_melt_map_racmo_firn_air_corrected.tif', engine = 'rasterio')

In [ ]:
meshx, meshy = np.meshgrid(img.x,img.y)

In [ ]:
meshlon, meshlat = uf.change_coord_stereo_to_latlon(meshx,meshy)

In [ ]:
img['longitude'] = xr.DataArray(data=meshlon, dims=['y','x'])
img['latitude'] = xr.DataArray(data=meshlat, dims=['y','x'])

In [ ]:
img.to_netcdf(inputpath_interim + 'Davison23_basal_melt_pattern.nc')

Create 1 km grid

In [ ]:
stereo_1km_grid = xr.open_dataset(inputpath_interim + 'Davison23_basal_melt_pattern.nc')

In [ ]:
# Create 2D-coordinates
meshx,meshy = np.meshgrid(stereo_1km_grid.x,stereo_1km_grid.y)

In [ ]:
# Stereographic resolution in m divided by 2 - in this case, the resolution is 2 km
half_res = 500. 

In [ ]:
# Create the lat and lon of the grid corners : ll = lower left, lr = lower right, ur = upper right, ul = upper left
meshlon_ll,meshlat_ll = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy - half_res)
meshlon_lr,meshlat_lr = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy - half_res)
meshlon_ur,meshlat_ur = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy + half_res)
meshlon_ul,meshlat_ul = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy + half_res)

In [ ]:
# Create the lat and lon of the grid centers 
meshlon,meshlat = uf.change_coord_stereo_to_latlon(meshx, meshy)

In [ ]:
# Formatting the dataset
lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nvertex','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nvertex','y','x'])
lon = xr.DataArray(data=np.array(meshlon), dims=['y','x'])
lat = xr.DataArray(data=np.array(meshlat), dims=['y','x'])

stereo_1km_grid = xr.merge([lon_bnds.rename('lon_bnds'),
                               lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat}) 

stereo_1km_grid['lon'].attrs['standard_name'] = 'longitude'
stereo_1km_grid['lon'].attrs['units'] = 'degrees_east'
stereo_1km_grid['lon'].attrs['bounds'] = 'lon_bnds'

stereo_1km_grid['lat'].attrs['standard_name'] = 'latitude'
stereo_1km_grid['lat'].attrs['units'] = 'degrees_north'
stereo_1km_grid['lat'].attrs['bounds'] = 'lat_bnds'

In [ ]:
# Write to netcdf
stereo_1km_grid[['lon','lat',
                    'lon_bnds','lat_bnds']].set_coords(('lat_bnds', 
                                                        'lon_bnds')).transpose('y','x','nvertex').to_netcdf(inputpath_interim + 'stereo_1km_withbnds.nc')

In [ ]:
6435*6120

In [ ]:
with open(inputpath_interim+"stereo_grid_1km_curvilinear_definition.txt", "w") as txt_file:

    txt_file.write('# \n')
    txt_file.write('# gridID 1 \n')
    txt_file.write('# \n')
    txt_file.write('gridtype  = curvilinear \n')
    txt_file.write('gridsize  = 39382200 \n')
    txt_file.write('datatype  = float \n')
    txt_file.write('xsize     = 6120 \n')
    txt_file.write('ysize     = 6435 \n')
    txt_file.write('xname     = lon \n')
    txt_file.write('xdimname  = x \n')
    txt_file.write('xlongname = "longitude" \n')
    txt_file.write('xunits    = "degrees_east" \n')
    txt_file.write('yname     = lat \n')
    txt_file.write('ydimname  = y \n')
    txt_file.write('ylongname = "latitude" \n')
    txt_file.write('yunits    = "degrees_north" \n')
    txt_file.write('vdimname  = nv4 \n')
    txt_file.write('xvals = ')
    for vv in tqdm(meshlon.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('yvals = ')
    for vv in tqdm(meshlat.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('xbounds = ')
    for vv in tqdm(stereo_1km_grid['lon_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('ybounds = ')
    for vv in tqdm(stereo_1km_grid['lat_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('# \n')
    txt_file.write('# gridID 2 \n')
    txt_file.write('# \n')
    txt_file.write('gridtype = projection \n')
    txt_file.write('gridsize  = 39382200 \n')
    txt_file.write('xsize     = 6120 \n')
    txt_file.write('ysize     = 6435 \n')
    txt_file.write('xname     = x \n')
    txt_file.write('xlongname = "x-coordinate in Cartesian system" \n')
    txt_file.write('xunits = "m" \n')
    txt_file.write('yname     = y \n')
    txt_file.write('ylongname = "y-coordinate in Cartesian system" \n')
    txt_file.write('yunits = "m" \n')
    txt_file.write('xfirst = '+str(stereo_1km_grid.x.min().values)+' \n')
    txt_file.write('xinc = 2000 \n')
    txt_file.write('yfirst = '+str(stereo_1km_grid.y.min().values)+' \n')
    txt_file.write('yinc = 2000 \n')
    txt_file.write('grid_mapping = Projection \n')
    
txt_file.close()

Remap it to NEMO grid

In [ ]:
# ncatted -O -a ,global,d,, stereo_1km_withbnds.nc stereo_1km_withbnds.nc
# ncap2 -O -s dummy[y,x]=1b stereo_1km_withbnds.nc stereo_1km_withbnds.nc
# ncatted -a coordinates,dummy,c,c,"lon lat" -a units,lon,c,c,"degreeE" -a units,lat,c,c,"degreeN" stereo_1km_withbnds.nc

In [6]:
cdo.remapbil(inputpath_mask_interim + 'NEMO_grid_withbnds.nc', input=inputpath_interim + 'Davison23_basal_melt_pattern_withgrid.nc', output = inputpath_interim + 'Davison23_basal_melt_pattern_on_eORCA1.nc')

'/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/Davison23_basal_melt_pattern_on_eORCA1.nc'

In [1]:
basal_melt_on_eORCA1 = xr.open_dataset('/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/Davison23_basal_melt_pattern_on_eORCA1.nc')

NameError: name 'xr' is not defined

In [ ]:
basal_melt_on_eORCA1['basal_melt'].plot(vmax=20)